In [1]:
import os
import numpy as np

from dotenv import load_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

from tqdm import tqdm



load_dotenv("../.env")


embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY"),
    chunk_size=1,
    dimensions=1024,
    show_progress_bar=False
)

vectore_store_yandex = Chroma(
    collection_name="ml_yandex_handbook",
    embedding_function=embeddings,
    persist_directory="./chroma_yandex_db"
)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [2]:
# Create retriever
retriever_yandex = vectore_store_yandex.as_retriever()

# Prompt
prompt = ChatPromptTemplate.from_template("""Generate the questions for the interviewee based on the selected topic of the interview:
{input}
And provided context (it's text of the book):

<context>
{context}
</context>

Генерировать надо на русском. Нужно сгенерировать как вопрос, так и ответ на этот вопрос, используя лишь приведенную информацию. В формате

Q: ...
A: ...

Всего нужно сгенерировать 10 вопросов.
""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_yandex, document_chain)
response = retrieval_chain.invoke({"input": "Глубокое обучение"})
print(response["answer"])

Q: Что такое глубинное обучение и как оно связано с нейронными сетями?  
A: Глубинное обучение — это область машинного обучения, основанная на нейронных сетях, которая стремится к обучению представлений объектов на основе данных. Нейронные сети, начиная с 2012 года, становятся стандартом в различных приложениях благодаря своей способности обучаться на больших объемах данных и автоматизировать процесс отбора признаков.

Q: Какие две ключевые идеи лежат в основе глубинного обучения?  
A: Первая идея — это переход к end-to-end обучению всей системы, что позволяет обучать все слои нейросети одновременно. Вторая идея — это обучение представлений объектов, что позволяет автоматически извлекать информативные признаки из данных, часто неразмеченных.

Q: Почему современные нейронные сети сложнее своих предшественников?  
A: Современные нейронные сети значительно сложнее из-за роста производительности компьютеров, увеличения объемов доступных данных и потребностей индустрии, что привело к более 

In [4]:
import re

generated_questions = response["answer"]

# Regular expressions to match questions and answers
questions = re.findall(r'Q: (.*?)\n', generated_questions, re.DOTALL)
answers = re.findall(r'A: (.*?)(?:\nQ:|\n?$)', generated_questions, re.DOTALL)

# Output results
print("Questions:")
print(len(questions))
print("\nAnswers:")
print(len(answers))

Questions:
10

Answers:
10


In [5]:
for question, answer in zip(questions, answers):
    print(question.strip())
    print(answer.strip())
    print()

Что такое глубинное обучение и как оно связано с нейронными сетями?
Глубинное обучение — это область машинного обучения, основанная на нейронных сетях, которая стремится к обучению представлений объектов на основе данных. Нейронные сети, начиная с 2012 года, становятся стандартом в различных приложениях благодаря своей способности обучаться на больших объемах данных и автоматизировать процесс отбора признаков.

Какие две ключевые идеи лежат в основе глубинного обучения?
Первая идея — это переход к end-to-end обучению всей системы, что позволяет обучать все слои нейросети одновременно. Вторая идея — это обучение представлений объектов, что позволяет автоматически извлекать информативные признаки из данных, часто неразмеченных.

Почему современные нейронные сети сложнее своих предшественников?
Современные нейронные сети значительно сложнее из-за роста производительности компьютеров, увеличения объемов доступных данных и потребностей индустрии, что привело к более сложным архитектурам и м

In [13]:
curr_question = questions[4]
curr_answer = answers[4]

my_answer = "Dropout - это техника регуляризации нейронных сетей, при которой рандомные нейроны выбрасываются с фиксированной вероятностью"
curr_answer.strip(), my_answer

('Dropout — это техника регуляризации, при которой случайным образом "выключаются" некоторые нейроны во время обучения, что помогает предотвратить переобучение и заставляет модель использовать все доступные параметры, а не полагаться на небольшое подмножество.',
 'Dropout - это техника регуляризации нейронных сетей, при которой рандомные нейроны выбрасываются с фиксированной вероятностью')

In [16]:
answer_llm = ChatOpenAI(
    model="gpt-4o-mini"
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты помогающий интервьюеру ассистент, который оценивает ответ собеседующего от 0 до 1 (вещественным числом) " + \
            "Дан следующий вопрос: {input_question} и вот его истинный ответ: {input_answer}. " + \
            "Если что можешь использовать свой предобученный контекст чтобы узнать дополнительные детали для ответа на вопрос." + \
            "Помимо этого нужно помочь собеседуемому понять, почему его оценка справедлива и какие детали еще необходимо упомянуть для полноценного ответа" + \
            "Необходимо дать ответ в формате: \n\n Grade: ... \n Tips: ...",
        ),
        ("human", "Вот ответ собеседуемого: {input}"),
    ]
)

chain = prompt | answer_llm

ai_answer_rate = chain.invoke(
    {
        "input_question": curr_question,
        "input_answer": curr_answer,
        "input": my_answer,
    }
)

ai_answer_rate.content

'Grade: 0.7  \nTips: Ваш ответ правильно определяет, что такое dropout и что это техника регуляризации, но он не совсем полон. Было бы полезно упомянуть, как именно dropout помогает предотвратить переобучение, например, за счет того, что модель становится менее зависимой от конкретных нейронов и вынуждена использовать все доступные параметры. Также стоит отметить, что dropout применяется только во время обучения, а во время тестирования все нейроны используются. Это добавило бы глубины вашему ответу.'

In [17]:
print(ai_answer_rate.content)

Grade: 0.7  
Tips: Ваш ответ правильно определяет, что такое dropout и что это техника регуляризации, но он не совсем полон. Было бы полезно упомянуть, как именно dropout помогает предотвратить переобучение, например, за счет того, что модель становится менее зависимой от конкретных нейронов и вынуждена использовать все доступные параметры. Также стоит отметить, что dropout применяется только во время обучения, а во время тестирования все нейроны используются. Это добавило бы глубины вашему ответу.


In [18]:
import re

# Regular expressions to extract grade and tips
grade_match = re.search(r'Grade:\s([\d.]+)', ai_answer_rate.content)
tips_match = re.search(r'Tips:\s(.+)', ai_answer_rate.content, re.DOTALL)

# Extracting and converting the grade
grade = float(grade_match.group(1)) if grade_match else None

# Extracting the tips
tips = tips_match.group(1).strip() if tips_match else None

# Output results
print("Grade:", grade)
print("\nTips:", tips)

Grade: 0.7

Tips: Ваш ответ правильно определяет, что такое dropout и что это техника регуляризации, но он не совсем полон. Было бы полезно упомянуть, как именно dropout помогает предотвратить переобучение, например, за счет того, что модель становится менее зависимой от конкретных нейронов и вынуждена использовать все доступные параметры. Также стоит отметить, что dropout применяется только во время обучения, а во время тестирования все нейроны используются. Это добавило бы глубины вашему ответу.
